In [1]:
# force the notebook to auto reload external python modules | useful for development
%load_ext autoreload
%autoreload 2

In [2]:
# imports
import sys

sys.path.append("..\\")
from src.data import MyDataset, AudioTripletDataset, LandmarkTripletDataset, AudioLandmarkTripletDataset
from src.features import TripletGenerator
# from src.models import main

import torch
from torch.utils.data import random_split
from torch.utils.data import DataLoader

random_seed = 42

In [3]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [4]:
# paths
raw_data_path = "..\\data\\raw\\"
processed_data_path = "..\\data\\processed\\"

---

In [5]:
my_dataset = MyDataset(location=raw_data_path)

In [ ]:
# create & pickle dataset | RUN THIS CELL ONLY ONCE TO CREATE THE DATASET
my_dataset.create_dataset()
my_dataset.save_dataset(processed_data_path)

In [6]:
# load dataset | RUN THIS CELL TO LOAD THE SAVED DATASET
my_dataset.load_dataset(processed_data_path)
sample_size = len(my_dataset)
print(f"Loaded dataset with {sample_size} samples.")

Loaded dataset with 323 samples.


In [ ]:
# train-valid-test split
# to maintain independence between train and test sets, split the dataset before creating the triplets

train_size = int(0.6 * sample_size)
val_size = int(0.2 * sample_size)
test_size = sample_size - train_size - val_size

train_set, val_set, test_set = random_split(my_dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(random_seed))

train_size, valid_size, test_size = len(train_set), len(val_set), len(test_set)
print(f"Train dataset size: {train_size} \nValid dataset size: {valid_size} \nTest dataset size: {test_size}")

In [7]:
# For Stratified Split

from sklearn.model_selection import train_test_split

# Assuming my_dataset is an instance of a Dataset class
sample_size = len(my_dataset)
targets = [my_dataset[i][1] for i in range(sample_size)]  # Assuming the second element is the label

# Splitting the data into train, validation, and test with stratification
train_idx, temp_idx, train_targets, temp_targets = train_test_split(
    range(sample_size), targets, test_size=0.2, stratify=targets, random_state=random_seed
)

val_idx, test_idx, val_targets, test_targets = train_test_split(
    temp_idx, temp_targets, test_size=0.2, stratify=temp_targets, random_state=random_seed
)

# Creating subsets
train_set = torch.utils.data.Subset(my_dataset, train_idx)
val_set = torch.utils.data.Subset(my_dataset, val_idx)
test_set = torch.utils.data.Subset(my_dataset, test_idx)

# Sizes of the datasets
train_size, valid_size, test_size = len(train_set), len(val_set), len(test_set)
print(f"Train dataset size: {train_size} \nValid dataset size: {valid_size} \nTest dataset size: {test_size}")


Train dataset size: 258 
Valid dataset size: 52 
Test dataset size: 13


In [8]:
# CREATE & PICKLE triplets of indicies | RUN THIS CELL ONLY ONCE TO CREATE THE TRIPLETS
# ensure diversity of triplets by setting num_triplets to a large number
# essentially 10 to 20 triplets per sample

num_triplets = 20

# TRAIN
train_triplets = TripletGenerator(train_set, num_triplets=train_size*num_triplets, prefix="train")
train_triplets.save_triplets(processed_data_path)

# VALID
valid_triplets = TripletGenerator(val_set, num_triplets=valid_size*num_triplets, prefix="valid")
valid_triplets.save_triplets(processed_data_path)

# TEST
test_triplets = TripletGenerator(test_set, num_triplets=test_size*num_triplets, prefix="test")
test_triplets.save_triplets(processed_data_path)

print(f"Created and saved: \
      \n {len(train_triplets.triplets)} train triplets \
      \n {len(valid_triplets.triplets)} valid triplets \
      \n {len(test_triplets.triplets)} test triplets"
      )

Created and saved:       
 5160 train triplets       
 1040 valid triplets       
 260 test triplets


---

In [7]:
# LOAD triplets | RUN THIS CELL TO LOAD THE SAVED TRIPLETS

# TRAIN
train_triplets = TripletGenerator(load=True, root_path=processed_data_path, prefix="train")
train_AL_triplets_dataset = AudioLandmarkTripletDataset(my_dataset.data, train_triplets.triplets)
train_audio_triplet_dataset = AudioTripletDataset(my_dataset.data, train_triplets.triplets)
train_landmark_triplet_dataset = LandmarkTripletDataset(my_dataset.data, train_triplets.triplets)

# VALID
valid_triplets = TripletGenerator(load=True, root_path=processed_data_path, prefix="valid")
valid_AL_triplets_dataset = AudioLandmarkTripletDataset(my_dataset.data, valid_triplets.triplets)
valid_audio_triplet_dataset = AudioTripletDataset(my_dataset.data, valid_triplets.triplets)
valid_landmark_triplet_dataset = LandmarkTripletDataset(my_dataset.data, valid_triplets.triplets)

# TEST
test_triplets = TripletGenerator(load=True, root_path=processed_data_path, prefix="test")
test_AL_triplets_dataset = AudioLandmarkTripletDataset(my_dataset.data, test_triplets.triplets)
test_audio_triplet_dataset = AudioTripletDataset(my_dataset.data, test_triplets.triplets)
test_landmark_triplet_dataset = LandmarkTripletDataset(my_dataset.data, test_triplets.triplets)

print(f"Loaded: \
      \n {len(train_triplets.triplets)} train triplets \
      \n {len(valid_triplets.triplets)} valid triplets \
      \n {len(test_triplets.triplets)} test triplets"
      )

Loaded:       
 5160 train triplets       
 1040 valid triplets       
 260 test triplets


In [8]:
# imports
from src.config import SystemConfig, TrainingConfig
from src.logging import setup_log_directory
from src.models import main
from src.visualization import plot_loss_accuracy

# logging
from torch.utils.tensorboard import SummaryWriter

# model summary & graph
from torchsummary import summary
from torchview import draw_graph
import graphviz
graphviz.set_jupyter_format('png')

In [9]:
def train_model(
        train_dataset: MyDataset, 
        valid_dataset: MyDataset, 
        model: torch.nn.Module,
        model_name: str
    ):
    
    # create dataloaders
    # Note to self: pin_mem for faster data transfer to GPU
    train_loader = DataLoader(
        train_dataset, 
        batch_size=TrainingConfig.batch_size, 
        shuffle=True, 
        pin_memory=True,
        num_workers=TrainingConfig.num_workers
    )
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=TrainingConfig.batch_size,
        shuffle=True,
        pin_memory=True,
        num_workers=TrainingConfig.num_workers
    )

    # train model
    summary(model, input_size=(TrainingConfig.batch_size, 250)) # needs input shape?
    model_graph = draw_graph(model, input_size=(TrainingConfig.batch_size, 250), expand_nested=True)
    model_graph.save(f"..\\reports\\figures\\{model_name}_model_graph.png")
                             
    model_graph.visual_graph

    training_config = TrainingConfig()

    # Model checkpoint log dir setup.
    training_config, current_version_name = setup_log_directory(training_config)

    # Tensorboard log dir setup.
    summary_writer = SummaryWriter(training_config.log_dir)

    # Train and Validate
    train_loss, train_acc, val_loss, val_acc = main(
        model,
        (train_loader, valid_loader),
        summary_writer=summary_writer,
        scheduler=None,
        system_config=SystemConfig(),
        training_config=training_config,
        data_augmentation=False,
        model_name=model_name,
    )

    # plot loss and accuracy
    plot_loss_accuracy(
        train_loss=[train_loss],
        val_loss=[val_loss],
        train_acc=[train_acc],
        val_acc=[val_acc],
        colors=["blue"],
        loss_legend_loc="upper center",
        acc_legend_loc="upper left",
    )
    

### Audio model

In [12]:
# importing models from src.models
from src.models import LSTM, LSTM_II, LSTM_III, CNN

# MODEL TO USE
init_model = LSTM_III

In [14]:
audio_data_shape=(250, 400)
model = init_model(input_shape=audio_data_shape)
train_model(
    train_audio_triplet_dataset, 
    valid_audio_triplet_dataset, 
    model,
    "audio_model.pt"
)

LSTM_III(
  (lstm): LSTM(400, 64, batch_first=True)
  (batch_norm_lstm): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dense1): Linear(in_features=16000, out_features=128, bias=True)
  (batch_norm_dense1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense2): Linear(in_features=128, out_features=64, bias=True)
  (batch_norm_dense2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense3): Linear(in_features=64, out_features=32, bias=True)
)
Logging at: ..\output\Logs_Checkpoints\Model_logs\version_32
Model Checkpoint at: ..\output\Logs_Checkpoints\Model_checkpoints\version_32


Train:	Epoch: 1/20 Train Loss: 0.4339, Train Acc: 0.5322: 100%|██████████| 162/162 [00:38<00:00,  4.19it/s]
Valid:	Epoch: 1/20 Valid Loss: 1.1686, Valid Acc: 0.0391:   6%|▌         | 2/33 [00:00<00:03,  9.17it/s]

pos_correct: 3, neg_correct: 31
pos_correct: 2, neg_correct: 31


Valid:	Epoch: 1/20 Valid Loss: 1.1203, Valid Acc: 0.0430:  12%|█▏        | 4/33 [00:00<00:03,  8.95it/s]

pos_correct: 4, neg_correct: 28
pos_correct: 2, neg_correct: 31


...

pos_correct: 2, neg_correct: 23
pos_correct: 4, neg_correct: 29


Valid:	Epoch: 17/20 Valid Loss: 1.5623, Valid Acc: 0.1212: 100%|██████████| 33/33 [00:05<00:00,  6.44it/s]


pos_correct: 9, neg_correct: 27
pos_correct: 3, neg_correct: 10



Train:	Epoch: 18/20 Train Loss: 0.0073, Train Acc: 0.7560:  63%|██████▎   | 102/162 [00:33<00:19,  3.06it/s]


KeyboardInterrupt: 

### Landmark Model

In [10]:
from src.models import STGCN

In [11]:
model = STGCN(num_nodes=20, in_channels=250, out_channels=64, num_frames=250)
train_model(
    train_landmark_triplet_dataset, 
    valid_landmark_triplet_dataset, 
    model,
    "landmark_model.pt"
)

STGCN(
  (temporal_conv1): Conv2d(250, 64, kernel_size=(3, 1), stride=(1, 1))
  (graph_conv1): GraphConvLayer(
    (fc): Linear(in_features=64, out_features=32, bias=True)
  )
  (temporal_conv2): Conv2d(32, 64, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
)
Logging at: ..\output\Logs_Checkpoints\Model_logs\version_42
Model Checkpoint at: ..\output\Logs_Checkpoints\Model_checkpoints\version_42


Train:	Epoch: 1/20 Train Loss: 1.0716, Train Acc: 31.8896: 100%|██████████| 162/162 [00:10<00:00, 15.61it/s]
Valid:	Epoch: 1/20 Valid Loss: 1.0035, Valid Acc: 32.0299:  33%|███▎      | 11/33 [00:00<00:00, 104.10it/s]

pos_correct: 2012, neg_correct: 77
pos_correct: 2040, neg_correct: 32
pos_correct: 2003, neg_correct: 113
pos_correct: 1891, neg_correct: 104
pos_correct: 1877, neg_correct: 206
pos_correct: 1908, neg_correct: 141
pos_correct: 1964, neg_correct: 89
pos_correct: 1903, neg_correct: 137
pos_correct: 1865, neg_correct: 191
pos_correct: 1854, neg_correct: 93
pos_correct: 2040, neg_correct: 168
pos_correct: 1887, neg_correct: 139
pos_correct: 1785, neg_correct: 207
pos_correct: 1806, neg_correct: 193
pos_correct: 1824, neg_correct: 124
pos_correct: 1959, neg_correct: 61
pos_correct: 1892, neg_correct: 308
pos_correct: 1857, neg_correct: 103
pos_correct: 1973, neg_correct: 138
pos_correct: 1826, neg_correct: 164
pos_correct: 1916, neg_correct: 87
pos_correct: 1887, neg_correct: 173
pos_correct: 1908, neg_correct: 223


Valid:	Epoch: 1/20 Valid Loss: 1.0017, Valid Acc: 32.0750: 100%|██████████| 33/33 [00:00<00:00, 108.45it/s]


pos_correct: 1967, neg_correct: 79
pos_correct: 1898, neg_correct: 144
pos_correct: 1953, neg_correct: 76
pos_correct: 1984, neg_correct: 97
pos_correct: 1845, neg_correct: 214
pos_correct: 2009, neg_correct: 47
pos_correct: 1957, neg_correct: 72
pos_correct: 2006, neg_correct: 103
pos_correct: 1860, neg_correct: 161
pos_correct: 1023, neg_correct: 73

Model Improved... Saving Model ... Done.




...

pos_correct: 2048, neg_correct: 0
pos_correct: 2048, neg_correct: 0
pos_correct: 2048, neg_correct: 0
pos_correct: 2048, neg_correct: 0
pos_correct: 2048, neg_correct: 0
pos_correct: 2048, neg_correct: 0
pos_correct: 2048, neg_correct: 0
pos_correct: 2048, neg_correct: 0
pos_correct: 2048, neg_correct: 0
pos_correct: 2048, neg_correct: 0
pos_correct: 2048, neg_correct: 0


Valid:	Epoch: 14/20 Valid Loss: 1.0000, Valid Acc: 32.0000: 100%|██████████| 33/33 [00:00<00:00, 53.29it/s]


pos_correct: 2048, neg_correct: 0
pos_correct: 2048, neg_correct: 0
pos_correct: 2048, neg_correct: 0
pos_correct: 2048, neg_correct: 0
pos_correct: 2048, neg_correct: 0
pos_correct: 2048, neg_correct: 0
pos_correct: 2048, neg_correct: 0
pos_correct: 2048, neg_correct: 0
pos_correct: 2048, neg_correct: 0
pos_correct: 1024, neg_correct: 0



Train:	Epoch: 15/20 Train Loss: 1.0000, Train Acc: 32.0000:  86%|████████▌ | 139/162 [00:03<00:00, 40.69it/s]


KeyboardInterrupt: 

### Combined model testing

In [ ]:
# import
from src.models import test, load_model


In [ ]:
# load models
audio_model = load_model("audio_model.pt")
landmark_model = load_model("landmark_model.pt")

In [ ]:
test(
    audio_model,
    landmark_model,
    test_AL_triplets_dataset,
)